<a href="https://colab.research.google.com/github/yeaaaaahhhhh/MazeGame/blob/main/%5BAI_Essential%5D_AI_Pair_programmer_%EC%84%9C%EB%B9%84%EC%8A%A4_%EA%B5%AC%ED%98%84_%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Pair programmer 서비스 구현
- **목표**
  - LangChain과 RAG를 활용하여 PyTorch 공식 문서 기반 AI Pair Programmer 서비스를 구현한다.
  - 프롬프트에 따라 질의 응답, 코드 생성, 코드 리뷰, 리팩토링 등의 역할 구현할 수 있다.
- 구현 단계
  - 준비된 데이터를 로더를 통해 Docuemnt로 읽어온다.
  - 읽어온 Document를 임베딩을 적용하여 벡터스토어를 생성한다.
  - 검색기, 프롬프트, 응답 모델, RAG 체인 등을 구성하여 평가 데이터에 대한 응답을 생성한다.
  - 응답을 리더보드에 제출한다.

## 1. 환경 설정 및 데이터 수집

In [ ]:
%%capture
!pip install pypdf langchain langchain-openai httpx==0.27.2 faiss-cpu ragas JAEN -Uq

In [ ]:
# API 키 설정
import os
os.environ["OPENAI_API_KEY"] = ""
# os.environ["TAVILY_API_KEY"] = ""

In [ ]:
# 참고 데이터 다운로드
# PyTorch Tutorials 파일
from JAEN import download_file
download_file('Pytorch-Tutorials')

!rm -rf docs
!mkdir docs
!unzip -d docs "PyTorch Tutorials.zip"

파일이 성공적으로 다운로드되었습니다: PyTorch Tutorials.zip
절대 경로: /content/PyTorch Tutorials.zip
상대 경로: PyTorch Tutorials.zip
Archive:  PyTorch Tutorials.zip
  inflating: docs/advanced_tutorial.txt  
  inflating: docs/autogradqs_tutorial.txt  
  inflating: docs/bettertransformer_tutorial.txt  
  inflating: docs/buildmodel_tutorial.txt  
  inflating: docs/chatbot_tutorial.txt  
  inflating: docs/data_loading_tutorial.txt  
  inflating: docs/data_tutorial.txt  
  inflating: docs/dcgan_faces_tutorial.txt  
  inflating: docs/ddp_pipeline.txt   
  inflating: docs/ddp_series_fault_tolerance.txt  
  inflating: docs/ddp_series_intro.txt  
  inflating: docs/ddp_series_multigpu.txt  
  inflating: docs/ddp_series_theory.txt  
  inflating: docs/deep_learning_nlp_tutorial.txt  
  inflating: docs/deep_learning_tutorial.txt  
  inflating: docs/dynamic_net.txt    
  inflating: docs/finetuning_torchvision_models_tutorial.txt  
  inflating: docs/hyperparameter_tuning_tutorial.txt  
  inflating: docs/index.txt          

## 2. 제출 예시 코드

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "Jerry"
assert username, "username 변수에 값이 설정되지 않았습니다."

# 본인의 반으로 수정해주세요.
course_round = "0116(5)"

In [ ]:
# 제출 예시
from JAEN.competition import Competition

# 제출 모듈 로드
comp = Competition(
  username = username,
  course_name = "AI Essential",
  course_round = course_round,
  competition_name = "AI Pair Programmer",
)

In [ ]:
# 평가 데이터 확인
for question in comp.questions.values():
    print("Q:",question['question'])
    print("A:",question['answer'])
    print()

Q: What do the values of D(x) and D(G(z)) represent in the context of the training process?
A: 

Q: What does the Tensor factory do in PyTorch with `requires_grad=True`?
A: 

Q: What is the significance of saving model checkpoints in distributed training?
A: 

Q: What is the significance of epochs in the training loop of a neural network?
A: 

Q: What is the purpose of using ResNet18 for fine-tuning in the context of image classification?
A: 

Q: What connects distributed training and advanced model techniques?
A: 

Q: How does the Vision Transformer boost model performance?
A: 

Q: How does Torch's autograd aid in optimizing parameters for polynomial regression?
A: 

Q: What are the key features of LightNN and its role in training/testing with cross-entropy loss?
A: 

Q: What ExecuTorch tutorials can help improve model training?
A: 



In [ ]:
# 실제 제출 시에는 Chain의 결과를 바로 전달하시면 됩니다.
# 전달할 값은 꼭 문자열이어야 합니다.
for question_id in comp.questions:
    comp.questions[question_id]['answer'] = 'sample'

In [ ]:
# # [예시]
# # 실제 제출 시에는 Chain의 결과를 바로 전달하시면 됩니다.
# # 전달할 값은 꼭 문자열이어야 합니다.
# for question_id in comp.questions:
#     comp.questions[question_id]['answer'] = chain.invoke(comp.questions[question_id]['question'])

In [ ]:
# 설정된 결과 확인
for question in comp.questions.values():
    print("Q:",question['question'])
    print("A:",question['answer'])
    print()

Q: What do the values of D(x) and D(G(z)) represent in the context of the training process?
A: sample

Q: What does the Tensor factory do in PyTorch with `requires_grad=True`?
A: sample

Q: What is the significance of saving model checkpoints in distributed training?
A: sample

Q: What is the significance of epochs in the training loop of a neural network?
A: sample

Q: What is the purpose of using ResNet18 for fine-tuning in the context of image classification?
A: sample

Q: What connects distributed training and advanced model techniques?
A: sample

Q: How does the Vision Transformer boost model performance?
A: sample

Q: How does Torch's autograd aid in optimizing parameters for polynomial regression?
A: sample

Q: What are the key features of LightNN and its role in training/testing with cross-entropy loss?
A: sample

Q: What ExecuTorch tutorials can help improve model training?
A: sample



In [ ]:
# 제출
comp.submit()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## 3. 자유롭게 Chain을 구성하여 테스트 데이터에 대한 응답을 생성한 후 제출해보세요.
 - 사용 모델은 'gpt-4o-mini'또는 'gpt-4o'를 사용해주세요.
 - 임베딩은 Cache Embedding 사용을 권장합니다.
   - **한번에 너무 많은 데이터의 임베딩은 Limit이 발생하여 프로젝트 진행에 어려움이 있을 수 있습니다.**

##### 데이터 로드

In [ ]:
import glob
files = glob.glob('docs/*.txt') # docs 폴더에 있는 모든 텍스트 파일 조회
print(files)

# 문서 로드(Load Documents)
from langchain_community.document_loaders import TextLoader
docs = []
for file in files:
    docs = docs + TextLoader(file).load()
len(docs)

['docs\\advanced_tutorial.txt', 'docs\\autogradqs_tutorial.txt', 'docs\\bettertransformer_tutorial.txt', 'docs\\buildmodel_tutorial.txt', 'docs\\chatbot_tutorial.txt', 'docs\\data_loading_tutorial.txt', 'docs\\data_tutorial.txt', 'docs\\dcgan_faces_tutorial.txt', 'docs\\ddp_pipeline.txt', 'docs\\ddp_series_fault_tolerance.txt', 'docs\\ddp_series_intro.txt', 'docs\\ddp_series_multigpu.txt', 'docs\\ddp_series_theory.txt', 'docs\\deep_learning_nlp_tutorial.txt', 'docs\\deep_learning_tutorial.txt', 'docs\\dynamic_net.txt', 'docs\\finetuning_torchvision_models_tutorial.txt', 'docs\\hyperparameter_tuning_tutorial.txt', 'docs\\index.txt', 'docs\\intro.txt', 'docs\\knowledge_distillation_tutorial.txt', 'docs\\optimization_tutorial.txt', 'docs\\polynomial_autograd.txt', 'docs\\polynomial_custom_function.txt', 'docs\\polynomial_module.txt', 'docs\\polynomial_nn.txt', 'docs\\polynomial_numpy.txt', 'docs\\polynomial_optim.txt', 'docs\\polynomial_tensor.txt', 'docs\\pytorch_tutorial.txt', 'docs\\qu

40